# <a id='toc1_'></a>[**MICRO-452 Final Report**](#toc0_)
### <a id='toc1_1_1_'></a>[*Group 45: Anton Balykov, Teo Halevi, Cyprien Lacassagne, Seonmin Park*](#toc0_)

# <a id='toc2_'></a>[Table of Contents](#toc0_)


- [Project Objective](#toc3_)    
- [I. Computer Vision](#toc4_)    
  - [I.1 Introduction](#toc4_1_)    
  - [I.2 Camera Set Up](#toc4_2_)    
  - [I.3 Global Obstables and Thymio Detection](#toc4_3_)    
- [II. Global Navigation](#toc5_)    
  - [II.1 Introduction](#toc5_1_)    
  - [II.2 Path Planning](#toc5_2_)    
  - [II.3 Algorithm](#toc5_3_)    
- [III. Local Navigation](#toc6_)    
  - [III.1 Introduction](#toc6_1_)    
  - [III.1 Local Avoidance](#toc6_2_)    
- [IV. Filtering](#toc7_)    
  - [IV.1 Introduction](#toc7_1_)    
  - [IV.2 Methodology](#toc7_2_)    
  - [IV.3 Dynamics of Thymio](#toc7_3_)    
  - [IV.4 Design and Implementation of Filter](#toc7_4_)    
  - [IV.5 Simulation](#toc7_5_)    
- [Conclusion](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc3_'></a>[Project Objective](#toc0_)

This project leverages the Thymio robot to explore and integrate key concepts introduced in the Introduction to Mobile Robotics course. We hereby focus on implementing and demonstrating core components: computer vision, global navigation, motion control, local navigation, and filtering. The mission of the robot is to navigate itself from the designated start position to the specified goal position, successfully avoiding fixed obstacles as well as dynamically placed one. Achieving this requires precise motion control, robust obstacle detection, and efficient filtering technique. Such practical application of the five above-mentioned elements would reflect our team's comprehensive understanding of mobile robotics principles.

# <a id='toc4_'></a>[I. Computer Vision](#toc0_)

## <a id='toc4_1_'></a>[I.1 Introduction](#toc0_)

bla bla bla

## <a id='toc4_2_'></a>[I.2 Camera Set Up](#toc0_)

bla bla bla

## <a id='toc4_3_'></a>[I.3 Global Obstables and Thymio Detection](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc5_'></a>[II. Global Navigation](#toc0_)

## <a id='toc5_1_'></a>[II.1 Introduction](#toc0_)

One main component of this project is navigation. Here, we discuss about our implementation of the global navigation. The main objectives of the global navigation is to find a collision-free path from a source and a goal, on a map filled with polygons acting as obtacles. The robot has to reach the goal while avoiding the obstacles. The strategy relies on a metric map and visibility graph.

## <a id='toc5_2_'></a>[II.2 Path Planning](#toc0_)

The class `Navigation` is constructed by receiving the position of the robot (source), the goal, and the corners of each obstacle. The latter are detected using polygon approximation in the class `ComputerVision`. For each obstacle, they are grouped into a list of points, representing its respective vertices.<br>

However, before going into graph creation, the polygons need to be expanded in order to account for the size of the robot. Otherwise, the robot would be considered as a single point object and bump into the obstacles.

Then, a visibility graph is built using the open-source library `pyvisgraph`.
Finally, the path is computed using the function `graph.get_shortest_path`, and returned to the main function to assess the robot's deviation from the goal.

## <a id='toc5_3_'></a>[II.3 Algorithm](#toc0_)

The obstacles are expanded in an intuitive manner: for each vertex, a point outside of the polygon is computed based on the direction of the two adjacent edges. The distance of the expansion is simply equals to the radius (diagonal) of the robot.<br>

The graph is constructed using a basic graph algorithm. Then calculations are made to filter out the connections violating free-space movement, i.e. connections between two vertices going through one or more obstacles.<br>
This yields a visibility graph connecting nodes that will form candidate paths from source to goal.
Then Dijkstra's algorithm is then used to find the shortest path in the graph.


# <a id='toc6_'></a>[III. Local Navigation](#toc0_)

## <a id='toc6_1_'></a>[III.1 Introduction](#toc0_)

bla bla bla

## <a id='toc6_2_'></a>[III.1 Local Avoidance](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc7_'></a>[IV. Filtering](#toc0_)

## <a id='toc7_1_'></a>[IV.1 Introduction](#toc0_)


There is a need for a method that compensates the defect that there is no direct mapping between Thymio's sensors and the values which are used for control. The absolute position of the robot can be obtained directly from the camera and global path planning techniques, but the robot may have low precision and update rate, causing drifts when the velocity or acceleration is integrated. We can address this challenge by implementing Kalman filter as it provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.

The Kalman filter we implemented estimates the position and speed of the robot when it moves along the planned path. Specifically, we implemented extended Kalman filter to take into account the system involves nonlinear dynamics that would need to be linearized around the current estimate, focusing on local optimality.


## <a id='toc7_2_'></a>[IV.2 Methodology](#toc0_)

Prior to the designing of our filter algorithm, we began with estimating Thymio's speed by let it cross seven blocks that are 50 mm each with a sample rate of 0.1 seconds and marked peaks of the ground sensor measurement indicating that it has crossed every block. This was to obtain velocity parameters, in which the method is identical to one of the activities found in Exercise 8 of the course. With known and obtained information, we could obtain the conversion factor of 0.4 to convert Thymio's speed to mm/s.

![Find Peaks](report-images/find_peaks.png "Fine Peaks")

Now, to obtain speed variance, we observed the data where the robot was moving by plotting the robot's speed and computed average velocity and thus the speed variance by diving the average speed with the robot's speed in mm/s.

![Measured Vel](report-images/measured_vel.png "Measured Vel")

With the assumption that half of the variance is cause by the measurement and the other half is caused by the perturbation to the states, we obtain 

$q_v = r_v = 16.08621$ $mm^2/s^2$ $\space$ where $\space$ $\sigma^2_{vel} = 32.17242$ $mm^2/s^2$

Variances on position state and measurement as well as those for the angle for orientation ($\sigma^2_{pos}$ and $\sigma^2_{\theta}$) were an arbitrary value that were tuned manually for the specific Thymio robot that is going to be used for demonstration.

## <a id='toc7_3_'></a>[IV.3 Dynamics of Thymio](#toc0_)

To begin with the implementation, the state space model is

$\hat{x}_{k-1|k-1} =  \begin{bmatrix}
                        x_{pos_{k-1}} \\
                        y_{pos_{k-1}} \\
                        \theta_{k-1} \\
                        v_{left_{k-1}} \\
                        v_{right_{k-1}} \\
                        \end{bmatrix}$

Under control input vector,

$ u =[v_{trans}, v_{rot}] $, $\space$ where $\space$ $v_{tran} = \frac{v_{left} + v_{right}}{2} $ $\space$ and $\space$ $ v_{rot} = \frac{v_{left} - v_{right}}{l_{base}} $

<img src="report-images/Thymio_odometry.png" width="571"/>

Then, we can see that the robot's new position and orientation are predicted in the following way

$x_{pos_{k}} = x_{pos_{k-1}} + v_{trans_{k}} \cdot \cos(\theta_{k-1}) \cdot dk$

$y_{pos_{k}} = y_{pos_{k-1}} + v_{trans_{k}} \cdot \sin(\theta_{k-1}) \cdot dk$

$\theta_{k} = \theta_{k-1} + v_{rot_{k}} \cdot dk$

## <a id='toc7_4_'></a>[IV.4 Design and Implementation of Filter](#toc0_)



Before diving into the steps of the extended Kalman filter we implemented, we would like to disclaim that the notations and formulas were referenced from lecture slides 8 of the Mobile Robotics course and online tutorial at [Extended Kalman Filter (EKF) with Python Code Example](https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/).

We have introduced that the extended Kalman filter takes into account nonlinear system dynamics in the previous sections. In order to make prediction of the next state, We use a nonlinear motion model $f(x, u)$. That is, we can formulate their relation as:

$\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k)$

Our $f(x, u)$ is then introduced as

$f(\hat{x}_{k-1|k-1}, u_k) = \begin{bmatrix}
1 & 0 & 0 & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & 0 & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -dk/l_{base} & dk/l_{base} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

In this prediction step of the filter, we make a use of the predicted covariance matrix $P_{k-1|k-1}$.

$P_{k-1|k-1} = \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

as each column corresponds to the state variables, from the first column to the last column, being $x_{pos_{k-1}}$, $y_{pos_{k_1}}$, $\theta_{k-1} $, $v_{left_{k-1}}$, and $v_{right_{k-1}}$, respectively.

The updated (current) covariance matrix is as follows:

$P_{k|k-1} = F_{k}P_{k-1|k-1}F_{k}^{T} + Q_{k}$

where $F_{k}$ is the Jacobian of $f_{k}$ matrix, and $Q_{k}$ is the noise covariance matrix of the state model. The $Q_{k}$ matrix specifically represents the degree or the extent of how much the actual motion of Thymio deviates from its assumed state space model which would be the path that it should follow for our case.

We can write the two matrices as below:

$F_{k|k-1} = 
\begin{bmatrix}
1 & 0 & -v_{trans} \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & v_{trans} \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -dk/l_{base} & dk/l_{base} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

$Q_{k|k-1} = \alpha \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

where $\alpha$ is a scaling factor, which we have initialized as $\alpha =10$, such that 

$Q_{k-1|k-1} = 10 \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

We set $\alpha$ large enough so that the filter tracks large changes in the wheel sensor measurements more closely as if it was smaller.

In [ ]:
def prediction(self, dt):
        f, F = self.compute_fnF(dt)
        self.x = f @ self.x
        self.P = F @ self.P @ F.T + self.Q

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


In [ ]:
def update(self, position, u, cam):
        if cam:
            R = self.R_cam
            H = self.H_cam
            z = np.array([position[0], position[1], position[2], u[0], u[1]])
        else:
            R = self.R_nocam
            H = self.H_nocam
            z = np.array([u[0], u[1]])

        z_pred = H @ self.x
        y = z - z_pred

        S = H @ self.P @ H.T + R
        K = self.P @ H.T @ np.linalg.pinv(S)
        self.x += K @ y
        self.P = self.P - K @ H @ self.P

The above steps are implemented as a class `KalmanFilterExtended` and the class is called in the `main.py` file as
```python
    ekf = KalmanFilterExtended(np.array([thymio_pos_x * -1, thymio_pos_y, ((thymio_theta + 180) % 360) * np.pi / 180]), u)
```

The function `run_EKF` ensures that the extened filter is intiated the filter steps are correctly called in the correct order. The code snippet below is part of it.

```python
    ekf.prediction(dt)
    measured_state = ekf.get_state()
    ekf.update([pos_x, pos_y, theta], u, cam)
    measurement_update = ekf.get_state()
``` 

It helps us to have the filter run in `main.py` in a loop throughout the entire mission of the robot as

```python
    measured_state, kidnap = run_EKF(ekf, thymio_pos_x * -1, thymio_pos_y, theta, u, cam=camera)
```

Additionally, we have set parameters that define *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. This mode is apart from the *kidnapping* condition from the Computer Vision side, and we decided to have two modes of kidnap detection to account for a variety of possible ways to kidnap the robot. Specifically, we have set the threshold for orientation and position to be 100 mm and 1 rad, and exceeding either (or both) of these values would be considered getting kidnaped.

```python
    if dpos > 100 or dtheta > 1:
        kidnap = True
        print("Thymio is being kidnapped (moved too far or rotated too much)")
        cur_t = time.time()
        ekf.count_time(cur_t)
        ekf.get_state()
    else:
        kidnap = False
``` 

We have also added a `draw_confidence_ellipse()` function that draws confidence ellipse taking from the first two diagonal components of the $P_{k}$ matrix as a means to visualize the uncertainty that the system has regarding the robot's pose.
```python
    P = get_cov().[:2, :2]
```

The method of implementing it was referenced from Github tutorial on [Robot Motion](https://github.com/jotaraul/jupyter-notebooks-for-robotics-courses/blob/master/3-Robot%20motion.ipynb) and [The Kalman Filter](https://engineeringmedia.com/controlblog/the-kalman-filter)

Then the extracted $P_{k}$ matrix will be

$P_{k} = \begin{bmatrix}
\sigma_{xx}^2 & \sigma_{xy}^2 \\ 
\sigma_{yx}^2 & \sigma_{yy}^2 \\
\end{bmatrix}$

We computed the eigenvalues and eigenvectors of it as the major and minor axis of the confidence ellipse is determined by the eigenvalues and the angles of the confidence ellipse is determined by the eigenvectors.

```python
    eigenvalues, eigenvectors = np.linalg.eigh(P)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
``` 

Then, we used ChatGPT to add chi-squared confidence interval to the ellipse computing function to account for the nonlinearity of the system.

```python
    chi_squared_val = np.sqrt(5.991) 
    amplification = 1
    major_axis = chi_squared_val * np.sqrt(eigenvalues[0]) * amplification
    minor_axis = chi_squared_val * np.sqrt(eigenvalues[1]) * amplification

    center = (int(mean[0]), int(mean[1]))
    axes = (int(major_axis), int(minor_axis))
    cv2.ellipse(frame, center, axes, angle, 0, 360, color, 2)
``` 
In `main.py`, it is called as
```python
    draw_confidence_ellipse(frame_aruco_and_corners, ekf.get_cov()[:2, :2], [x, y], color=(0, 255, 255))
```
in the loop of updating every measurement so that the ellipse is also plotted dynamically.
This visualization of positional uncertainty in 2D space provides better intuition of how the uncertainty changes during the mission and different modes such as kidnapping and switching between vision and no vision.

## <a id='toc7_5_'></a>[IV.5 Simulation](#toc0_)

The simulation on filtering was done several times on the path planning scenario found in the previous section of Global Navigation. One of the examples is shown below, where the `run_EKF` receives a set $x$ and $y$ coordinate of the planned path in each iteration and applies filtering from the `KalmanFilterExtended` class functions. The code snippet of this can be found in `KFsim.py`. The purpose of this simulation is to observe whether the filtering is successfully running throughout the entire travel sequence and to observe different results in two different conditions: when Thymio has vision and when it does not. The simulation is done by imposing random noises of normal distribution on the wheel sensors and camera vision and compare the path the robot would take with the true path the path planning provides. When the simulation confirms that the filtering is operating well, we could integrate the algorithm into our `main.py`.

<img src="report-images/EKF_simulation.png" width="571"/>
<img src="report-images/kalman_simu.png" width="571"/>

The second figure shows the confidence ellipses, where the purple ellipses represent with vision and the yellow ellipses represent without vision. For visualization purposes, the purple ellipses were amplified 150-fold and the yellow ellipses were amplified 50-fold. Even with higher amplification factor, the confidence ellipses are significantly smaller for with vision case than for the without vision case, which means that the system tend to have much more certainty when the camera is not covered than when the camera is covered. The yellow ellipses grow with time because the longer the robot is blocked from vision, the more uncertain further steps will be.

# <a id='toc8_'></a>[Conclusion](#toc0_)
It has been an incredible journey to embark on this project within the constrained timeline of just three weeks, replete with challenges and invaluable learning opportunities. From the very beginning, we have had to face numerous obstacles, such as technical errors and conceptual hurdles, which have pushed us to refine our problem-solving skills and deepen our understanding of mobile robotics. Despite these challenges, our team has been able to overcome setbacks and achieve results we are proud of through persistence, discipline, and a collaborative spirit.

Working through the complexities of the project was as rewarding as it was demanding. From understanding the theoretical basis to the implementation of practical solutions, each step provided an opportunity to reinforce and showcase our grasp of the principles of mobile robotics. The hands-on nature of the project tested not only our knowledge but also helped us grow as engineers, fostering resilience, adaptability, and creativity in tackling real-world problems.

We would like to express our deep gratitude to everyone who guided and supported us through this journey. We would like to thank Professor Mondada, who inspired us with his experience and vision, and the teaching assistants, who were always ready to help, explain doubts, and give us valuable insights. Their encouragement and mentorship played a very important role in driving the success of our project.

It is not only an academic milestone but also a source of personal and professional growth, and we will be carrying the lessons learned herein as we progress in our future endeavors related to robotics and beyond.